In [1]:
#Import all library needed
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

#confusion matrix visualization
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
data = pd.read_csv('heart.csv')

In [3]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
catagorialList = ['sex','cp','fbs','restecg','exang','ca','thal']
for item in catagorialList:
    data[item] = data[item].astype('object') #casting to object

In [5]:
data = pd.get_dummies(data, drop_first=True)

In [6]:
y = data['target'].values
y = y.reshape(y.shape[0],1)
x = data.drop(['target'],axis=1)

x.shape

(303, 21)

In [8]:
minx = np.min(x)
maxx = np.max(x)
x = (x - minx) / (maxx - minx)
x.head()

,age,trestbps,chol,thalach,oldpeak,slope,sex_1,cp_1,cp_2,cp_3,...,restecg_1,restecg_2,exang_1,ca_1,ca_2,ca_3,ca_4,thal_1,thal_2,thal_3
0,0.111702,0.257092,0.413121,0.265957,0.004078,0.000000,0.001773,0.000000,0.000000,0.001773,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.001773,0.000000,0.0
1,0.065603,0.230496,0.443262,0.331560,0.006206,0.000000,0.001773,0.000000,0.001773,0.000000,...,0.001773,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001773,0.0
2,0.072695,0.230496,0.361702,0.304965,0.002482,0.003546,0.000000,0.001773,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001773,0.0
3,0.099291,0.212766,0.418440,0.315603,0.001418,0.003546,0.001773,0.001773,0.000000,0.000000,...,0.001773,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001773,0.0
4,0.101064,0.212766,0.627660,0.289007,0.001064,0.003546,0.000000,0.000000,0.000000,0.000000,...,0.001773,0.0,0.001773,0.0,0.0,0.0,0.0,0.000000,0.001773,0.0


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
# re-create train and validation set
x_train, x_val, y_train, y_val  = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
# train 70%, validation 20%, test 10%
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(217, 21)
(55, 21)
(31, 21)


In [13]:
# Placeholder for the results
results = []

# Hyperparameters
optimizers = {'Adam': tf.keras.optimizers.Adam, 'Lion': tf.keras.optimizers.Lion}
learning_rates = [0.01, 0.001]
loss_functions = ['mse', 'binary_crossentropy']
epochs_list = [500, 1000]

# Iterate over all combinations of hyperparameters
for optimizer_name, optimizer in optimizers.items():
    for lr in learning_rates:
        for loss_function in loss_functions:
            for epochs in epochs_list:
                # Build the model
                model = Sequential()
                model.add(Dense(64, input_dim=21, activation='softmax'))
                model.add(Dense(32, activation='softmax'))
                model.add(Dense(1, activation='sigmoid'))

                # Compile the model
                model.compile(loss=loss_function, optimizer=optimizer(learning_rate=lr), metrics=['accuracy'])

                # Train the model
                model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=16, verbose=1)

                # Predict on test data
                y_pred = model.predict(x_test)
                y_pred = np.round(y_pred).astype(int)

                # Performance Evaluation
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)

                # Store the result
                result = {
                    'optimizer': optimizer_name, 'learning_rate': lr,
                    'loss_function': loss_function, 'epochs': epochs,
                    'accuracy': accuracy,
                    'precision': precision, 'recall': recall
                }
                results.append(result)

# Print the results
for res in results:
    print(f"Optimizer: {res['optimizer']}, Learning Rate: {res['learning_rate']}, Loss Function: {res['loss_function']}, Epochs: {res['epochs']}")
    print(f"Test Accuracy: {res['accuracy']}, Precision: {res['precision']}, Recall: {res['recall']}\n")


Epoch 1/500


14/14 [==============================] - 2s 30ms/step - loss: 0.2493 - accuracy: 0.5438 - val_loss: 0.2483 - val_accuracy: 0.5455
Epoch 2/500
14/14 [==============================] - 0s 8ms/step - loss: 0.2489 - accuracy: 0.5438 - val_loss: 0.2480 - val_accuracy: 0.5455
Epoch 3/500
14/14 [==============================] - 0s 7ms/step - loss: 0.2481 - accuracy: 0.5438 - val_loss: 0.2480 - val_accuracy: 0.5455
Epoch 4/500
14/14 [==============================] - 0s 7ms/step - loss: 0.2487 - accuracy: 0.5438 - val_loss: 0.2479 - val_accuracy: 0.5455
Epoch 5/500
14/14 [==============================] - 0s 9ms/step - loss: 0.2481 - accuracy: 0.5438 - val_loss: 0.2480 - val_accuracy: 0.5455
Epoch 6/500
14/14 [==============================] - 0s 7ms/step - loss: 0.2486 - accuracy: 0.5438 - val_loss: 0.2481 - val_accuracy: 0.5455
Epoch 7/500
14/14 [==============================] - 0s 9ms/step - loss: 0.2484 - accuracy: 0.5438 - val_loss: 0.2480 - val_accuracy: 0.5455
Epoch 8/5